# Reversal Symmetry for Simple Stable Voting

This notebook contains the code used to verify the reversal symmetry property for simple stable voting (SSV) as described in Section 5 of the paper "Stable Voting and the Splitting of Cycles".

In [1]:
%pip install python-sat
%pip install pref_voting

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pysat.formula import IDPool
from pysat.solvers import Solver
import itertools
import functools
from pref_voting.weighted_majority_graphs import MarginGraph
from pref_voting.margin_based_methods import split_cycle, stable_voting, simple_stable_voting

In [3]:
for n in range(2, 11):
    print(f"Encoding {n}", flush=True)
    
    pool = IDPool()
    s = Solver(use_timer=True)
    
    # "precedes" variable definitions
    def lt(a, b, c, d, reverse):
        if reverse:
            return lt(c, d, a, b, False)
        if a == b or c == d:
            raise ValueError(f'Edges may not be self-loops.')
        if a == c and b == d:
            raise ValueError('Edges must be distinct.')
        if (a > d) or (a == d and b > c):
            return lt(d, c, b, a, False)
        return pool.id(f'{a}{b}{c}{d}')
    
    # "SSV" variable definitions
    def SSV(V, x, reverse):
        return pool.id(f"{x}win{''.join(str(v) for v in V)}{"rev" if reverse else ""}")
    
    # all undirected edges between a collection of nodes 
    def edges(V=range(n)):
        return itertools.permutations(V, 2)
    
    def ordering():
        added = set()
        for ((a, b), (c, d), (e, f)) in itertools.permutations(edges(), 3):
            first = lt(a, b, c, d, False)
            second = lt(c, d, e, f, False)
            if (first, second) not in added:
                added.add((first, second))
                s.add_clause([-first, -second, lt(a, b, e, f, False)])
        for ((a, b), (c, d)) in itertools.permutations(edges(), 2):
            s.add_clause([lt(a, b, c, d, False), lt(b, a, d, c, False)])
            s.add_clause([-lt(a, b, c, d, False), -lt(b, a, d, c, False)])
    
    def removed(V, x):
        return tuple(v for v in V if v != x)
    
    def SSV_conditions():
        for rev in [False, True]:
            for V in itertools.chain.from_iterable(itertools.combinations(range(n), r) for r in range(1, n+1)):
                s.add_clause([SSV(V, x, rev) for x in V])
        
                for (x, y) in edges(V):
                    s.add_clause([-SSV(V, x, rev), -SSV(V, y, rev)])
                
                for (a, b) in edges(V):
                    clause = [-SSV(removed(V, b), a, rev), SSV(V, a, rev)]
                    for (c, d) in edges(V):
                        if c != a or d != b:
                            # tseitin iff -SSV((removed(V, d), c)) \/ lt(c, d, a, b)
                            tseitin = pool.id(f"t{a}{b}{c}{d}in{''.join(str(x) for x in V)}{"rev" if rev else ""}")
                            s.add_clause([-SSV(removed(V, d), c, rev), lt(c, d, a, b, rev), -tseitin])
                            s.add_clause([SSV(removed(V, d), c, rev), tseitin])
                            s.add_clause([-lt(c, d, a, b, rev), tseitin])
                            clause.append(-tseitin)
                    s.add_clause(clause)
    
    
    def symmetry_breaking():
        for i in range(n):
            V = tuple(range(i+1))
            s.add_clause([SSV(V, 0, False)])
            for (c, d) in edges(V):
                if c != 0 or d != i:
                    s.add_clause([-SSV(removed(V, d), c, False), lt(c, d, 0, i, False)])
    
    
    def check_model(model):
        def compare(x, y):
            (a, b) = x
            (c, d) = y
            return 1 if lt(a, b, c, d, False) in model else -1 # descending
        edge_list = sorted(edges(), key=functools.cmp_to_key(compare))
        max_weight = (n * (n - 1) // 2)
        return MarginGraph(list(range(n)), 
                           [(edge_list[i][0], edge_list[i][1], max_weight - i) for i in range(len(edge_list) // 2)])
    
    
    # add the clauses
    ordering()
    SSV_conditions()
    symmetry_breaking()                             
    s.add_clause([SSV(tuple(range(n)), 0, True)]) # 0 is also the winner in the reverse. 

    print(s.nof_clauses(), flush=True)
    print(f"Starting {n}", flush=True)
    
    if s.solve(): # takes around 45 seconds for n=7.
        result = check_model(s.get_model())
        result.display()
        print(f"Time: {s.time()}")
        print(f"Split Cycle Winners: {split_cycle(result)}")
        print(f"Stable Voting Winner: {stable_voting(result)}")
        print(f"Simple Stable Voting Winner: {simple_stable_voting(result)}", flush=True)
        break
    else:
        print(f"No counterexamples of size {n}", flush=True)

Encoding 2
22
Starting 2
No counterexamples of size 2
Encoding 3
444
Starting 3
No counterexamples of size 3
Encoding 4
3371
Starting 4
No counterexamples of size 4
Encoding 5
16441
Starting 5
No counterexamples of size 5
Encoding 6
62688
Starting 6
No counterexamples of size 6
Encoding 7
206970
Starting 7


error: Caught keyboard interrupt